In [1]:
import copy
import json
import logging
from collections import defaultdict
from typing import Any, Dict, List, Optional, Sequence, Tuple, TypeVar, Union

import math
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from allennlp.common.checks import ConfigurationError

In [ ]:
# torch.Size([4, 18, 3])
# targets shape torch.Size([4, 18, 3])
# logit flats shape torch.Size([72, 62])
# log_probs_flatshape torch.Size([72, 62])
# log_probs_flat_flat shape torch.Size([4464, 1])
# negative_log_likelihood_flat shape torch.Size([216, 1])
# negative_log_likelihood_flat tensor([[4.1174],
# negative_log_likelihood shape torch.Size([4, 18, 3])

In [ ]:
logits = torch.randn(2, 4, 3)
print(logits)
print(logits.shape)

In [ ]:
# flattens logits (batch_size * seq_len, num classes)
logits_flat = logits.view(-1, logits.size(-1))
print(logits_flat)
print(logits_flat.shape)

In [ ]:
# looks good, the probs are always highest where the logits are
log_probs_flat = torch.nn.functional.log_softmax(logits_flat, dim=-1)
print(log_probs_flat)
print(log_probs_flat.shape)

In [ ]:
# NB need to flatten again
log_probs_flat_flat = log_probs_flat.view(-1, 1)
print(log_probs_flat_flat)
print(log_probs_flat_flat.shape)

In [ ]:
z = torch.LongTensor([[[1., 0., 0.],
         [4., 0., 0.],
         [2., 3., 0.],
         [4., 2., 0.]],

        [[3., 2., 1.],
         [4., 3., 2.],
         [6., 4., 6.],
         [7., 7., 7.]]])

print(z.shape)
targets=z

In [ ]:
# label ids, e.g. ([32, 22, 3])
#targets = torch.LongTensor(2, 4, 3).random_(0, 10)
#targets = torch.randn(2, 4, 3)
# targets = torch.zeros(2, 4, 3)
# print(targets)
# print(targets.shape)

In [ ]:
# shape : (batch * max_len, 1)
targets_flat = targets.view(-1,targets.size(-1))#.long()

print(targets_flat)
print(targets_flat.shape)

In [ ]:
# targets are 3x longer because of 3 labels
# ???
negative_log_likelihood_flat = - torch.gather(log_probs_flat, dim=0, index=targets_flat)
print(negative_log_likelihood_flat)
print(negative_log_likelihood_flat.shape)

In [ ]:
negative_log_likelihood = negative_log_likelihood_flat.view(*targets.size())
print(negative_log_likelihood)
print(negative_log_likelihood.shape)

In [ ]:
tensor = torch.zeros(log_probs_flat.size())  # vector of zeros
tensor.shape

tensor.scatter_(0, torch.LongTensor(targets_flat), 1)

In [ ]:
# label padding
ymask = log_probs_flat_flat.data.new(log_probs_flat_flat.size()).zero_() # (N,C) all zero
print(ymask)
print(ymask.shape)

In [ ]:
ymask.scatter_(0, z.view(-1,1), 4) # have to make y into shape (N,1) for scatter_ to be happy
print(ymask)
ymask = Variable(ymask)
# pluck
logpy = (log_probs_flat_flat * ymask).sum(1) # this hurts in my heart

In [ ]:
logpy

In [ ]:
print(z.view(-1,1))

In [ ]:
# 2
logp = torch.nn.functional.log_softmax(logits_flat, dim=-1)
print(logp)


In [ ]:
logpy = torch.gather(logp, 0, targets_flat).view(-1)
print(logpy)

In [ ]:
inputs = Variable(torch.rand(5,2))
targets = Variable(torch.ones(5)).long()

print(inputs)
print(targets)

In [18]:
y_hat = torch.randn(8, 5)
#y_hat = torch.LongTensor(8, 5).random_(0, 10)
print(y_hat)
#y = torch.randn(16, 5)#.long()
y = torch.LongTensor(8, 5).random_(0, 10)
print(y)

tensor([[ 1.8606,  0.0039, -0.1796,  0.8380,  0.2205],
        [ 1.6247, -1.0459,  0.9171,  1.3358,  1.4977],
        [ 0.3386, -0.6297,  0.3660, -0.3239,  0.4321],
        [-0.1238, -0.9727,  1.7626,  0.8756, -0.4538],
        [ 0.1278,  0.1793,  0.8850, -0.6632, -1.3309],
        [ 0.2309,  0.9529, -1.9914,  0.1280, -0.3643],
        [ 0.4050, -0.7707,  0.4456,  0.1011,  0.7456],
        [-0.0980,  2.2740, -1.1081, -0.2956,  0.3958]])
tensor([[4, 7, 5, 7, 2],
        [7, 8, 3, 8, 2],
        [1, 1, 4, 5, 4],
        [9, 3, 9, 8, 1],
        [8, 0, 1, 7, 0],
        [9, 3, 4, 7, 1],
        [9, 6, 6, 8, 7],
        [7, 6, 5, 0, 3]])


In [19]:
logp = torch.nn.functional.log_softmax(y_hat, dim=-1) 
# get (N,C) log probabilities
print(logp)
print(logp.shape)

tensor([[-0.6097, -2.4664, -2.6499, -1.6323, -2.2497],
        [-1.1606, -3.8312, -1.8682, -1.4495, -1.2876],
        [-1.3915, -2.3598, -1.3642, -2.0540, -1.2981],
        [-2.4387, -3.2877, -0.5524, -1.4394, -2.7688],
        [-1.5833, -1.5318, -0.8261, -2.3743, -3.0420],
        [-1.5305, -0.8085, -3.7528, -1.6334, -2.1257],
        [-1.5027, -2.6784, -1.4620, -1.8066, -1.1621],
        [-2.6771, -0.3051, -3.6871, -2.8746, -2.1832]])
torch.Size([8, 5])


In [23]:
logp_flat = logp.view(-1, 1)
print(logp_flat)
print(logp_flat.shape)

tensor([[-0.6097],
        [-2.4664],
        [-2.6499],
        [-1.6323],
        [-2.2497],
        [-1.1606],
        [-3.8312],
        [-1.8682],
        [-1.4495],
        [-1.2876],
        [-1.3915],
        [-2.3598],
        [-1.3642],
        [-2.0540],
        [-1.2981],
        [-2.4387],
        [-3.2877],
        [-0.5524],
        [-1.4394],
        [-2.7688],
        [-1.5833],
        [-1.5318],
        [-0.8261],
        [-2.3743],
        [-3.0420],
        [-1.5305],
        [-0.8085],
        [-3.7528],
        [-1.6334],
        [-2.1257],
        [-1.5027],
        [-2.6784],
        [-1.4620],
        [-1.8066],
        [-1.1621],
        [-2.6771],
        [-0.3051],
        [-3.6871],
        [-2.8746],
        [-2.1832]])
torch.Size([40, 1])


In [21]:
y_flat = y.view(-1,1)
print(y_flat.shape)

torch.Size([40, 1])


In [27]:
# prepare an (N,C) array of 1s at the locations of ground truth
ymask = logp_flat.data.new(logp_flat.size()).zero_() # (N,C) all zero
#ymask = logp.data.new(logp.size()).zero_() # (N,C) all zero
print(ymask.shape)

ymask.scatter_(0, y.view(-1,1), 1) # have to make y into shape (N,1) for scatter_ to be happy


torch.Size([40, 1])


tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [32]:
#ymask = Variable(ymask)
# pluck
# multiply flattened logits by positive examples
logpy = (logp_flat * ymask).sum(1) # this hurts in my heart

In [33]:
logpy

tensor([-0.6097, -2.4664, -2.6499, -1.6323, -2.2497, -1.1606, -3.8312, -1.8682,
        -1.4495, -1.2876,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000])

In [ ]:
# label ids, e.g. ([32, 22, 3])
#targets = torch.LongTensor(2, 4, 3).random_(0, 10)
#targets = torch.randn(2, 4, 3)
# targets = torch.zeros(2, 4, 3)
# print(targets)
# print(targets.shape)